## Exploration of Cat Images




In [ ]:
# used to import YOLO used for cropping provided imgs
from ultralytics import solutions
import cv2
from ultralytics import YOLO
# used to resume training if anything is interupted. Commeted out
# while learning so no test_test data persists
# results = model.train(resume=True)

# used for parsing files given their directory
from pathlib import Path
import os
import shutil 

# used to encode cropped imgs into a vector of predictable length and size
import clip
import torch
from PIL import Image
import torchvision.transforms as transforms
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
# clip_model, clip_preprocess = clip.load('RN50', device=device)

from sklearn.model_selection import train_test_split


In [18]:
cat_names = ['mousse','sherbert']

MOVE_DIR = f'/home/kiki/yolo-project/datasets/train/images'
INPUT_IMAGES_PATH = f'/home/kiki/yolo-project/datasets/input_images/'

TRAIN_TXT_PATH = f'/home/kiki/yolo-project/datasets/train/labels/'
TRAIN_IMAGE_PATH = f'/home/kiki/yolo-project/datasets/train/images/'

VAL_TXT_PATH = f'/home/kiki/yolo-project/datasets/val/labels/'
VAL_IMAGE_PATH = f'/home/kiki/yolo-project/datasets/val/images/'

-- Util functions used for data cleaning. Would be kept in a seprate file normally. 

In [ ]:

def video_to_frames(input_loc, output_loc, catname):
    """
    Function to extract frames from an input video file and save them
    as separate frames in an output directory.
    """

    cap = cv2.VideoCapture(input_loc)
    if not cap.isOpened():
        print(f"Error: Could not open video {input_loc}")
        return

    frame_count = 0
    print("Converting video to frames...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_filename = os.path.join(output_loc, f"{catname}_{frame_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)
        
        frame_count += 1

    cap.release()
    print(f"Video capture released. Extracted {frame_count} frames.")

catname = 'mousse'
input_video = f'/home/kiki/yolo-project/video/mousse_03.mp4'
output_folder = TRAIN_IMAGE_PATH
video_to_frames(input_video, output_folder, catname)

Converting video to frames...
Video capture released. Extracted 650 frames.


In [ ]:
def rename_files(folder_path, prefix="", suffix="", start_number=1, keep_extension=True):
    """
    Rename all files in a folder with a pattern.
    Args:
        folder_path: Path to the folder containing files
        prefix: Text to add before the number (e.g., "image_")
        suffix: Text to add after the number (e.g., "_photo")
        start_number: Starting number for sequential numbering
        keep_extension: Whether to keep original file extensions
    """
    folder = Path(folder_path)
    
    if not folder.exists():
        print(f"Error: Folder '{folder_path}' does not exist")
        return
    
    files = [f for f in folder.iterdir() if f.is_file()]
    
    if not files:
        print("No files found in the folder")
        return
    
    files.sort()
    print(f"Found {len(files)} files to rename\n")
    
    temp_files = []
    for i, file in enumerate(files, start=start_number):
        temp_name = f"__temp_{i}_{file.name}"
        temp_path = folder / temp_name
        try:
            file.rename(temp_path)
            temp_files.append((temp_path, i))
        except Exception as e:
            print(f"✗ Error creating temp name for {file.name}: {e}")
            return
    
    for temp_path, i in temp_files:
        extension = temp_path.suffix if keep_extension else ""
        original_ext = temp_path.name.split('_', 3)[-1]
        if keep_extension:
            extension = Path(original_ext).suffix
        
        new_name = f"{prefix}{i}{suffix}{extension}"
        new_path = folder / new_name
        try:
            temp_path.rename(new_path)
            print(f"✓ Renamed: {temp_path.name.split('_', 3)[-1]} -> {new_name}")
        except Exception as e:
            print(f"✗ Error renaming {temp_path.name}: {e}")
    
    print("\nRenaming complete!")

if __name__ == "__main__":
    rename_files(
        folder_path=f'/home/kiki/yolo-project/datasets/train/images/sherbert',
        prefix="sherbert_",
        start_number=1
    )

In [ ]:

def rename_jpg_to_lowercase(folder_path):
    """
    Renames all .JPG files to .jpg in a folder.
    
    Args:
        folder_path: Path to the folder containing files
    """
    folder = Path(folder_path)
    
    if not folder.exists():
        print(f"Error: Folder '{folder_path}' does not exist")
        return
    
    jpg_files = list(folder.glob('*.JPG'))
    
    if not jpg_files:
        print("No .JPG files found")
        return
    
    print(f"Found {len(jpg_files)} .JPG files to rename\n")
    
    for file in jpg_files:
        new_name = file.stem + '.jpg'  
        new_path = file.parent / new_name
        
        try:
            file.rename(new_path)
            print(f"✓ Renamed: {file.name} -> {new_name}")
        except Exception as e:
            print(f"✗ Error renaming {file.name}: {e}")
    
    print(f"\nRenaming complete!")

# Usage:
rename_jpg_to_lowercase('/home/kiki/yolo-project/datasets/train/images')

-- Functions that are utilized by the model to train.

In [ ]:
def folder_to_array(given_image_path):

    """
    Adds all images in a given folder to an array
    and returns that array

    Args:
        given_image_path: the path given by the user
    """

    print('Reading all files from', given_image_path)
    
    file_list = []
    # Extracting all the contents in the directory corresponding to path
    l_files = os.listdir(given_image_path)
    # Iterating over all the files
    for i,file in enumerate(l_files):
        file_path = f'{given_image_path}{file}'
            # Checking whether the given file is a directory or not
        if not os.path.isfile(file_path):
            print(f'ALERT: {file} is not a file, so can not be appended!')
        else:
            try:
                file_list.append(file_path)
            except:
                    # Catching if any error occurs and alerting the user
                print(f'ALERT: {file} could not be found! Please check\
                    the associated softwares, or the file type.')
    return file_list

In [ ]:
def create_lables(img_file_arry, model, TRAIN_TXT_PATH):

    """
    Creates labels from the given imgs. Lables are saved in the designated
    dirctory and as .txt files.

    Args:
        img_file_arry: the given array of imgs. Accepted format types include JPG, jpg, jpeg, PNG, and TIFF.
            * All files must be of the same file extention
        model: the existing model. Used to create the tensors that will become the labels
        TRAIN_TXT_PATH: the path in which we want to save the labels
    """

    results = model(img_file_arry, stream=True)  # return a list of Results objects
    # Process results list
    for i, result in enumerate(results):
        save_txt = True
        start = 46
        # start = img_file_arry[i].find('mousse') if 'mousse' in img_file_arry[0] else img_file_arry[0].find('sherbert')
        # all files have the same path, and differ at the same point.
        # this point is 46 but to avoid magic numbers we'd IDEALLY run the above.
        #  for some reason, if i do is start to get errors. we love that. hard coding for now TwT
        end = img_file_arry[i].find('.')
        # similar to 'start' all file endings are three chars long. 
        # this point is the file len - 1 (str slicing excludes the end index) 
        # but to avoid magic numbers we run the above. 

        name = img_file_arry[i][start:end]

        if name == '':
            print(f"Failed to process {img_file_arry[i]}: naming error '' ")
            break
            
        try:
            result.save_txt(f"{TRAIN_TXT_PATH}{name}.txt")
            # print(f"{TRAIN_TXT_PATH}{name}.txt")
        except Exception as e:
            print(f"Failed to process {img_file_arry[i]}: {e}")
    
        

In [ ]:
def reclassify_lables(TRAIN_TXT_PATH):

    """
    Takes the generated labels and reclassifis them based on 
    interal variables.

    Args:
        TRAIN_TXT_PATH: the path in which we want to save the labels
    """


    l_files = os.listdir(TRAIN_TXT_PATH)
    # Iterating over all the files
    for file in l_files:
    # Instantiating the path of the file
        file_path = f'{TRAIN_TXT_PATH}{file}'
        # Checking whether the given file is a directory or not
        if not os.path.isfile(file_path):
            print(f'ALERT: {file} is not a file, so can not be printed!')
        else:
            # Read the file
            with open(file_path, 'r') as f:
                lines = f.readlines()
            # Change line
            for i in range(len(lines)):
                if '15' in lines[i][:3]:
                    left = lines[i][:3]
                    right = lines[i][3:]
                    if 'mousse' in file_path:
                        left = '0'
                    else:
                        left = '1'
                    lines[i] = ' '.join([left, right])
                else:
                    lines[i] = ''
            # write the changes lines
            with open(file_path, 'w') as f:
                f.writelines(lines)


In [ ]:
def is_file_empty(file_path):

  """
    Checks if a file is empty based on its size in bytes.

    Args:
        file_path : the path of the given file
    """
  # Returns True if file size is 0, False otherwise
  return os.path.getsize(file_path) == 0

In [ ]:
def remove_empty_labels(label_file_arry):

    """
    Removes empty label/.txt files to prevent false negatives

    Args:
        label_file_arry : the array of label files
    """

    for file_name in label_file_arry:
        if not os.path.exists(file_name):
            print(f"'{file_name}' not found.")
        elif is_file_empty(file_name):
            print(f"'{file_name}' is empty.")
            os.remove(file_name)
        else:
            print(f"'{file_name}' is not empty.")

In [ ]:
def clean_imgs_and_lables(img_file_arry, label_file_arry, MOVE_DIR):

    """
    Moves image files without matching label files to a given directory

    Args:
        img_file_arry : the array of image files
        label_file_arry : the array of label files
        MOVE_DIR : the target directory

    """

    img_set = set(img_file_arry)
    label_set = set(label_file_arry)

    for img_file in img_set:
        start = img_file.find('mousse') if 'mousse' in img_file else img_file.find('sherbert')
        end = img_file.find('.')
        for label_file in label_set:
            if img_file[start:end] == label_file[start:end]:
                try:
                    shutil.move(img_file, MOVE_DIR)
                    # print(f"Moved: {img_file} to {MOVE_DIR}")
                except Exception as e:
                    print(f"Skipped: {img_file} already exists. Earray Error: ", e)  
        
    print('Removing empty labels....')
    remove_empty_labels(label_file_arry)

In [ ]:
def move_file (file_array, destination):

    """
    Moves files in an array from their current directory to another

    Args:
        file_array : array of given files to move
        destination : target directory

    """

    for file_source in file_array:
        try:
            shutil.move(file_source, destination)
            print(f"Moved: {file_source} to {destination}")
        except Exception as e:
             print(f"Skipped: {file_source} already exists. Error: ", e)  
        

In [ ]:
def reset ():

    """
    Resets training data by moving all images back to the input path and 
    removes all generated labels.

    """

    print('resetting data....')

    print('moving all images back to',INPUT_IMAGES_PATH)

    temp_img_arr = folder_to_array(TRAIN_IMAGE_PATH)
    move_file (temp_img_arr, INPUT_IMAGES_PATH)

    temp_img_arr= folder_to_array(VAL_IMAGE_PATH)
    move_file (temp_img_arr, INPUT_IMAGES_PATH)

    print('deleting all labels....')
    for file in folder_to_array(TRAIN_TXT_PATH):
        os.remove(file)
    
    for file in folder_to_array(VAL_TXT_PATH):
        os.remove(file)

Reading all files from /home/kiki/yolo-project/datasets/train/images/
Reading all files from /home/kiki/yolo-project/datasets/val/images/


-- the code below will build and create model. UTIL functions will need to be typed in by user

In [ ]:
torch.cuda.empty_cache()

img_file_arry = folder_to_array(INPUT_IMAGES_PATH)

Reading all files from /home/kiki/yolo-project/datasets/input_images/


In [12]:
# Load a model
model = YOLO("yolo11n.pt")  # pretrained YOLO11n model

In [13]:
BATCH_SIZE = 64

In [ ]:

for start_index in range(0,len(img_file_arry),BATCH_SIZE):
    end_index = start_index + BATCH_SIZE
    create_lables(img_file_arry[start_index:end_index], model)

label_file_arry = folder_to_array()

In [ ]:
if len(img_file_arry) != len(label_file_arry):
    print(f'Moving images with labels to {TRAIN_IMAGE_PATH}')

    clean_imgs_and_lables(img_file_arry, label_file_arry)

    print('Recalculating image array...')

    img_file_arry = folder_to_array(TRAIN_IMAGE_PATH)

print(f'DEBUG STATEMENT :: array lens {len(img_file_arry)} == {len(label_file_arry)}')

In [16]:

reclassify_lables()

In [ ]:

# now to randomly sort and move imgs and txt files into /train or /val
X_train, X_test, y_train, y_test = train_test_split(
     img_file_arry, label_file_arry, test_size=0.2, random_state=42)

# x = imgs
# y = lables

move_file(X_test, VAL_IMAGE_PATH) 
move_file(y_test, VAL_TXT_PATH)

# Train the model
# results = model.train(data="cat_data.yaml", epochs=100, imgsz=640)

In [ ]:
# Load a model
model = YOLO("yolo11n.pt")  # pretrained YOLO11n model
results = model.train(data="cat_data.yaml", epochs=100, imgsz=640)

In [5]:
model = YOLO('/home/kiki/cat-cam/runs/detect/train7/weights/best.pt')

In [ ]:
model.predict(f'/home/kiki/Downloads/PXL_20251219_212417341.jpg')